In [1]:
from datasets import load_dataset

# Télécharger le dataset depuis Hugging Face
dataset = load_dataset("PedroSampaio/fruits-360")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 67690
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 22688
    })
})

In [3]:
# Hyperparamètres optimisés pour économiser les ressources
batch_size = 8  # Réduction de la charge mémoire
num_epochs = 5  # Moins d'itérations
learning_rate = 0.0005  # Plus petit pour un fine-tuning stable
image_size = 128  # Réduction de la taille des images

In [4]:
import torchvision.transforms as transforms

# Définir les transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
# Appliquer les transformations aux images
def transform_dataset(split):
    return [(transform(img["image"]), img["label"]) for img in dataset[split]]

train_data = transform_dataset("train")
test_data = transform_dataset("test")

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
from torch.utils.data import DataLoader

#Création des DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Vérification
images, labels = next(iter(train_loader))
print(f"Shape des train_images: {images.shape}")
print(f"train_labels: {labels[:5]}")

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Charger le modèle pré-entraîné ResNet50
#model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# Chargement du modèle ResNet50 pré-entraîné
model = models.resnet50(pretrained=True)

# ❄️ Geler toutes les couches sauf la dernière (économie de ressources)
for param in model.parameters():
    param.requires_grad = False

# Adapter la dernière couche pour Fruits-360 (nombre de classes à définir)
num_classes = 131 
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)


In [ ]:
import torch.optim as optim
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Entraînement avec consommation minimale
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Entraînement en half-precision si GPU
        with torch.cuda.amp.autocast() if scaler else torch.no_grad():
            outputs = model(images)
            loss = criterion(outputs, labels)

        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
# Évaluation du modèle
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# Sauvegarde du modèle optimisé
torch.save(model.state_dict(), "resnet50_light.pth")
print("Modèle sauvegardé!")

In [ ]:
#Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"\nTest Accuracy: {test_acc:.2%}")

In [ ]:
#Faire des prédictions
y_pred_probs = model.predict(test_images)
y_pred = np.argmax(y_pred_probs, axis=1) 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Matrice de confusion
conf_matrix = confusion_matrix(test_labels, y_pred)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Affichage de la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(131), yticklabels=range(131))
plt.xlabel("Prédictions")
plt.ylabel("Vérités")
plt.title("Matrice de Confusion")
plt.show()

In [ ]:
# 4. Rapport de classification
print("Rapport de Classification :")
print(classification_report(test_labels, y_pred, digits=2))

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

# Charger une nouvelle image
image_path = "istockphoto-184276818-612x612.jpg" 
image = Image.open(image_path)

# Appliquer les transformations (resize + normalisation)
image = image.resize((100, 100))  
image = np.array(image) / 255.0    
image = np.expand_dims(image, axis=0)  

# Faire la prédiction
y_pred_probs = model.predict(image)  
y_pred_class = np.argmax(y_pred_probs, axis=1)[0]  

# Afficher l’image et la classe prédite
plt.imshow(Image.open(image_path))
plt.axis("off")
plt.title(f"Prédiction : Classe {y_pred_class}")
plt.show()
